# Extract technical metadata

This recipe will extract technical metadata from a directory of datasets and export it to a CSV file. It requires the python libraries `pandas`, `geopandas`, and `rasterio`. Part 1 writes the filenames, coordinate reference system, file format, resource type, and (optionally) the WKT polygon outline. Part 2 creates CSV files of the attribute table field names and types.

Created 2024-10-18 by Karen Majewicz

In [ ]:
import os
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.warp import transform_bounds
from shapely.geometry import Polygon
from shapely.ops import transform
from shapely.geometry import box

## Part 1: Extract metadata to a CSV

### Setup CSV and directories

In [ ]:
# Define a mapping from variable names to desired column headers
column_mapping = {
    'folder_name': 'Folder Name',
    'filename': 'File Name',
    'crs': 'Conforms To',
    'file_format': 'Format',
    "total_area_km2": 'Extent',
    'spatial_resolution': 'Spatial Resolution',
    'geometry_type': 'Resource Type',
    'bounding_box': 'Bounding Box',
    'wkt_outline': 'Geometry',
    'folder_size': 'File Size'
    
}

# Define the root directory for the geospatial data
root_directory = 'data'

# Define the output directory for the attribute table CSV files
output_directory = 'codebooks'


## Functions

### File size

In [ ]:
# function to add up the files in each dataset folder

def get_folder_size(folder_path, unit='MB', decimal_places=3):
    """
    Calculate the total size of all files in a folder and return it in the specified unit.

    Parameters:
    - folder_path (str): Path to the folder.
    - unit (str): The unit for the size ('bytes', 'KB', 'MB'). Default is 'MB'.
    - decimal_places (int): The number of decimal places to round the size to. Default is 3.

    Returns:
    - float: Total size of the folder contents in the specified unit, rounded to the specified number of decimal places.
    """
    total_size = 0
    for dirpath, _, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # Add to the total size only if it is a file (not a broken link, etc.)
            if os.path.isfile(fp):
                total_size += os.path.getsize(fp)

    # Convert the total size to the specified unit
    if unit == 'KB':
        total_size /= 1024  # Convert bytes to kilobytes
    elif unit == 'MB':
        total_size /= (1024 * 1024)  # Convert bytes to megabytes

    # Round the total size to the specified number of decimal places
    rounded_size = round(total_size, decimal_places)
    return rounded_size

### Geometry type

In [ ]:
def process_geometry_type(data, is_raster=False):
    """
    Determine the geometry type of a GeoDataFrame or indicate if the dataset is a raster.

    Parameters:
    - data (GeoDataFrame or DatasetReader): The data source, which can be a GeoDataFrame for vector data or DatasetReader for raster.
    - is_raster (bool): Flag to indicate if the data source is a raster.

    Returns:
    - str: The geometry type description, or 'Unknown' if the geometry type cannot be determined.
    """
    if is_raster:
        return "Raster data"

    if data.empty or data.geometry.is_empty.all():
        return 'Unknown'

    try:
        # Get unique geometry types in the GeoDataFrame
        geometry_types = data.geom_type.unique()

        # Format the geometry type for output
        if len(geometry_types) == 1:
            # Single geometry type
            geometry_type = geometry_types[0].replace("LineString", "Line").replace("MultiPolygon", "Polygon")
        else:
            # Mixed geometry types
            geometry_type = "Mixed geometries"

        return f"{geometry_type} data"
    except Exception as e:
        print(f"Failed to determine geometry type: {e}")
        return 'Unknown'



### Report original CRS

In [ ]:
# function to reformat the CRS into a resolvable URI

def format_crs_uri(crs_string):
    # If the CRS is in the "EPSG:xxxx" format, convert it to a resolvable URI
    if crs_string and crs_string.startswith("EPSG:"):
        epsg_code = crs_string.split(":")[1]
        return f"https://epsg.io/{epsg_code}"
    else:
        # Return the original CRS string if it's not an EPSG code
        return crs_string

### Rounding function (check)

In [ ]:
def round_coordinates(geometry, decimal_places=2):
    """
    Round the coordinates of a geometry to the specified number of decimal places.

    Parameters:
    - geometry (Geometry): The input Shapely geometry.
    - decimal_places (int): Number of decimal places to round to.

    Returns:
    - Geometry: The geometry with rounded coordinates.
    """
    if geometry.is_empty:
        return geometry

    # Function to round coordinates
    def rounder(x, y, z=None):
        if z is None:
            return (round(x, decimal_places), round(y, decimal_places))
        else:
            return (round(x, decimal_places), round(y, decimal_places), round(z, decimal_places))

    # Apply the rounding function using transform
    return transform(rounder, geometry)

### Bounding box

In [ ]:
def calculate_bounding_box(gdf, decimal_places=4):
    """
    Calculate and format the bounding box for a GeoDataFrame in WGS84 (EPSG:4326).
    
    Parameters:
    - gdf (GeoDataFrame): The GeoDataFrame to process.
    - decimal_places (int, optional): Number of decimal places to round coordinates.

    Returns:
    - str: The formatted bounding box as a string.
    """
    if gdf.empty or gdf.crs is None:
        return 'Unknown'

    try:
        # Convert to WGS84 for bounding box calculation
        gdf = gdf.to_crs(epsg=4326)
        bounds = gdf.total_bounds
        rounded_bounds = [round(coord, decimal_places) for coord in bounds]
        return f"{rounded_bounds[0]},{rounded_bounds[1]},{rounded_bounds[2]},{rounded_bounds[3]}"
    except Exception:
        return 'Unknown'


def calculate_bounding_box_raster(src, decimal_places=4):
    """
    Calculate the bounding box and WKT outline for a raster file in WGS84 (EPSG:4326).
    
    Parameters:
    - src (rasterio.io.DatasetReader): The raster source.
    - decimal_places (int, optional): Number of decimal places to round coordinates.

    Returns:
    - tuple: A tuple containing the formatted bounding box as a string and the WKT representation of the bounding box.
    """
    if src.crs is None:
        return 'Unknown', 'None'

    try:
        # Reproject the bounding box to WGS84 if needed
        left, bottom, right, top = src.bounds
        if src.crs.to_string() != 'EPSG:4326':
            left, bottom, right, top = transform_bounds(src.crs, 'EPSG:4326', left, bottom, right, top)

        # Round the coordinates
        rounded_bounds = [round(coord, decimal_places) for coord in [left, bottom, right, top]]
        bbox_str = f"{rounded_bounds[0]},{rounded_bounds[1]},{rounded_bounds[2]},{rounded_bounds[3]}"

        # Create WKT for a Polygon representing the bounding box
        wkt_outline = f"POLYGON(({rounded_bounds[0]} {rounded_bounds[1]}, {rounded_bounds[0]} {rounded_bounds[3]}, " \
                      f"{rounded_bounds[2]} {rounded_bounds[3]}, {rounded_bounds[2]} {rounded_bounds[1]}, " \
                      f"{rounded_bounds[0]} {rounded_bounds[1]}))"

        return bbox_str, wkt_outline
    except Exception as e:
        print(f"Failed to calculate bounding box and WKT outline: {e}")
        return 'Unknown', 'None'


### Geometry (WKT Outline)

In [ ]:
def generate_wkt_outline(gdf, simplify_tolerance=None, decimal_places=2):
    """
    Generate a WKT representation of a generalized outline for the dataset.

    Parameters:
    - gdf (GeoDataFrame): The GeoDataFrame containing the geometries.
    - simplify_tolerance (float, optional): The tolerance for the simplify() method to reduce detail.
    - decimal_places (int, optional): The number of decimal places to round the coordinates.

    Returns:
    - str: The WKT representation of the generalized outline.
    """
    if gdf.empty or gdf.crs is None:
        return 'None'

    try:
        # Convert to WGS84 for WKT generation
        gdf = gdf.to_crs(epsg=4326)

        # Create a unified geometry from all geometries in the GeoDataFrame
        unified_geom = gdf.geometry.unary_union

        # Use the convex hull to create a generalized outline
        if not unified_geom.is_empty:
            generalized_outline = unified_geom.convex_hull
        else:
            return 'None'

        # Optionally simplify the outline for further generalization
        if simplify_tolerance is not None:
            generalized_outline = generalized_outline.simplify(simplify_tolerance)

        # Round the coordinates of the outline
        generalized_outline = round_coordinates(generalized_outline, decimal_places)

        # Convert the resulting geometry to WKT using shapely's wkt module
        if isinstance(generalized_outline, Polygon):
            wkt_outline = generalized_outline.wkt
        else:
            return 'None'

        return wkt_outline
    except Exception:
        return 'None'

def generate_raster_wkt(bbox, src_crs=None, decimal_places=2):
    """
    Generate a WKT representation of a bounding box for a raster, optionally transforming to WGS84.

    Parameters:
    - bbox (list): A list of bounding box coordinates [left, bottom, right, top].
    - src_crs (CRS, optional): The original CRS of the bounding box. If provided and not EPSG:4326, the bbox will be transformed.
    - decimal_places (int, optional): Number of decimal places to round coordinates.

    Returns:
    - str: The WKT representation of the bounding box as a Polygon.
    """
    left, bottom, right, top = bbox

    # If src_crs is provided and is not EPSG:4326, transform the bounding box
    if src_crs and src_crs.to_string() != 'EPSG:4326':
        left, bottom, right, top = transform_bounds(src_crs, 'EPSG:4326', left, bottom, right, top)

    # Round the coordinates
    left = round(left, decimal_places)
    bottom = round(bottom, decimal_places)
    right = round(right, decimal_places)
    top = round(top, decimal_places)

    # Create the WKT for a Polygon representing the bounding box
    wkt = f"POLYGON(({left} {bottom}, {left} {top}, {right} {top}, {right} {bottom}, {left} {bottom}))"
    return wkt



### Area

In [ ]:
def calculate_total_area(gdf):
    """
    Calculate the total area covered by a GeoDataFrame in square kilometers using an equal-area projection.

    Parameters:
    - gdf (GeoDataFrame): The GeoDataFrame to process.

    Returns:
    - float or str: The total area in square kilometers, or 'Unknown' if unavailable.
    """
    if gdf.empty or gdf.crs is None:
        return 'Unknown'

    try:
        # Reproject to an equal-area projection for accurate area calculation
        gdf = gdf.to_crs(epsg=6933)
        total_area_km2 = gdf.geometry.area.sum() / 1e6  # Convert to square kilometers
        return round(total_area_km2, 3)
    except Exception:
        return 'Unknown'
    
def calculate_total_area_raster(src):
    """
    Calculate the total area covered by a raster file in square kilometers.

    Parameters:
    - src (rasterio.io.DatasetReader): The raster source.

    Returns:
    - float or str: The total area in square kilometers, or 'Unknown' if unavailable.
    """
    if src.crs is None:
        return 'Unknown'

    try:
        # Reproject to an equal-area projection for accurate area calculation
        gdf = gdf.to_crs(epsg=6933)
        # Calculate area based on pixel resolution and total number of pixels
        pixel_size_x, pixel_size_y = src.res  # Resolution of each pixel in meters
        total_pixels = src.width * src.height  # Total number of pixels in the raster
        total_area_m2 = pixel_size_x * pixel_size_y * total_pixels  # Total area in square meters
        total_area_km2 = total_area_m2 / 1e6  # Convert to square kilometers

        return round(total_area_km2, 3)
    except Exception as e:
        print(f"Failed to calculate total area for raster: {e}")
        return 'Unknown'

### Spatial resolution

In [ ]:
# VECTOR: Calculate average distance between vertices


def calculate_avg_vertex_distance(geom):
    """
    Calculate the average distance between consecutive vertices of a polygon, line geometry, or multi-part geometry.

    Parameters:
    - geom (shapely.geometry): A Shapely geometry object (Polygon, LineString, MultiPolygon, or MultiLineString).

    Returns:
    - float: The average distance between consecutive vertices, or None if not applicable.
    """
    if geom.is_empty or not geom.is_valid:
        return None
    
    # Handle multi-part geometries by iterating through each sub-geometry
    if geom.geom_type.startswith('Multi'):
        all_distances = []
        for part in geom.geoms:
            part_distance = calculate_avg_vertex_distance(part)
            if part_distance is not None:
                all_distances.append(part_distance)
        # Calculate the mean of the distances if there are any valid distances
        if all_distances:
            return sum(all_distances) / len(all_distances)
        else:
            return None

    # Handle single-part geometries
    coords = list(geom.exterior.coords) if geom.geom_type == 'Polygon' else list(geom.coords)
    
    if len(coords) < 2:
        return None
    
    # Calculate the distance between each consecutive pair of coordinates
    distances = [
        ((coords[i][0] - coords[i-1][0]) ** 2 + (coords[i][1] - coords[i-1][1]) ** 2) ** 0.5
        for i in range(1, len(coords))
    ]
    
    # Return the average distance
    return sum(distances) / len(distances)

def calculate_spatial_resolution_vector(gdf):
    """
    Calculate the average spatial resolution (vertex distance) for a GeoDataFrame using an equal-area projection.

    Parameters:
    - gdf (GeoDataFrame): The GeoDataFrame to process.

    Returns:
    - float or str: The spatial resolution in meters, or 'Unknown' if unavailable.
    """
    if gdf.empty or gdf.crs is None:
        return 'Unknown'

    try:
        # Reproject to an equal-area projection for accurate spatial resolution calculation
        gdf = gdf.to_crs(epsg=6933)
        avg_vertex_distance = gdf.geometry.apply(calculate_avg_vertex_distance)
        if avg_vertex_distance.notna().any():
            return round(avg_vertex_distance.dropna().mean(), 3)
        else:
            return 'Unknown'
    except Exception:
        return 'Unknown'


In [ ]:
def extract_metadata(directory, simplify_tolerance=None, include_wkt=True, decimal_places=2):
    """
    Extract metadata from geospatial datasets in a directory.

    Parameters:
    - directory (str): The directory containing the datasets.
    - simplify_tolerance (float, optional): Tolerance for simplifying WKT outlines.
    - include_wkt (bool, optional): Whether to include the WKT outline in the metadata.
    - decimal_places (int, optional): Number of decimal places to round WKT coordinates.

    Returns:
    - None
    """
    metadata = []  # List to hold metadata for each file

    # Supported vector formats by GeoPandas
    vector_formats = {'.shp': 'Shapefile', '.geojson': 'GeoJSON'}

    # Walk through the directory
    for root, _, files in os.walk(directory):
        for filename in files:
            file_ext = os.path.splitext(filename)[1].lower()
            filepath = os.path.join(root, filename)
            folder_name = os.path.basename(os.path.dirname(filepath))
            folder_size = get_folder_size(os.path.dirname(filepath), unit='MB')

            # Vector Data Processing
            if file_ext in vector_formats:
                process_vector(filepath, filename, vector_formats[file_ext], folder_name, folder_size, metadata,
                               simplify_tolerance, include_wkt, decimal_places)

            # Raster Data Processing
            elif file_ext == '.tif':
                process_raster(filepath, filename, folder_name, folder_size, metadata, include_wkt, decimal_places)

        # Geodatabase Processing (directories with .gdb)
        if root.endswith('.gdb'):
            process_geodatabase(root, folder_name, folder_size, metadata, simplify_tolerance, include_wkt, decimal_places)

    # Convert metadata to DataFrame and save as CSV
    df = pd.DataFrame(metadata)
    output_csv = os.path.join(directory, 'geospatial_metadata.csv')
    df.to_csv(output_csv, index=False)
    print(f'Metadata extraction complete. CSV saved to {output_csv}')


def process_vector(filepath, filename, file_format, folder_name, folder_size, metadata, simplify_tolerance, include_wkt, decimal_places):
    try:
        gdf = gpd.read_file(filepath)
        original_crs = gdf.crs.to_string() if gdf.crs else 'Unknown'
        crs_uri = format_crs_uri(original_crs) if gdf.crs else 'Unknown'

        # Calculate metadata components
        bbox = calculate_bounding_box(gdf, decimal_places)
        wkt_outline = generate_wkt_outline(gdf, simplify_tolerance, decimal_places) if include_wkt else None
        total_area_km2 = calculate_total_area(gdf)  # Use vector area calculation
        spatial_resolution = calculate_spatial_resolution_vector(gdf)

        # Process geometry type
        geometry_type = process_geometry_type(gdf)

        # Store metadata
        file_metadata = {
            'filename': filename,
            'folder_name': folder_name,
            'crs': crs_uri,
            'file_format': file_format,
            'geometry_type': geometry_type,
            'bounding_box': bbox,
            'total_area_km2': total_area_km2,
            'spatial_resolution': spatial_resolution,
            'folder_size': f"{folder_size} MB",
            'wkt_outline': wkt_outline if include_wkt else None
        }
        metadata.append(file_metadata)

    except Exception as e:
        print(f"Could not read vector file {filename}: {e}")



def process_raster(filepath, filename, folder_name, folder_size, metadata, include_wkt, decimal_places):
    try:
        with rasterio.open(filepath) as src:
            original_crs = src.crs.to_string() if src.crs else 'Unknown'
            crs_uri = format_crs_uri(original_crs) if src.crs else 'Unknown'

            # Calculate spatial resolution and area
            pixel_size_x, pixel_size_y = src.res
            spatial_resolution = round((pixel_size_x + pixel_size_y) / 2, 3)
            total_area_km2 = calculate_total_area_raster(src)  # Use raster area calculation

            # Get bounding box and WKT outline
            bbox, wkt_outline = calculate_bounding_box_raster(src, decimal_places)

            # Store metadata
            file_metadata = {
                'filename': filename,
                'folder_name': folder_name,
                'crs': crs_uri,
                'file_format': 'GeoTIFF',
                'geometry_type': 'Raster data',
                'bounding_box': bbox,
                'total_area_km2': total_area_km2,
                'spatial_resolution': spatial_resolution,
                'folder_size': f"{folder_size} MB",
                'wkt_outline': wkt_outline if include_wkt else None
            }
            metadata.append(file_metadata)

    except Exception as e:
        print(f"Could not read raster file {filename}: {e}")


# Helper function for geodatabase processing
def process_geodatabase(root, folder_name, folder_size, metadata, simplify_tolerance, include_wkt, decimal_places):
    try:
        layers = gpd.io.file.fiona.listlayers(root)
        for layer in layers:
            gdf = gpd.read_file(root, layer=layer)
            original_crs = gdf.crs.to_string() if gdf.crs else 'Unknown'
            crs_uri = format_crs_uri(original_crs) if gdf.crs else 'Unknown'

            # Calculate metadata components
            bbox = calculate_bounding_box(gdf, decimal_places)
            wkt_outline = generate_wkt_outline(gdf, simplify_tolerance, decimal_places) if include_wkt else None
            total_area_km2 = calculate_total_area(gdf)
            spatial_resolution = calculate_spatial_resolution_vector(gdf)

            # Process geometry type
            geometry_type = process_geometry_type(gdf)

            # Store metadata
        file_metadata = {
            'filename': f"{os.path.basename(root)} - {layer}",
            'folder_name': folder_name,
            'crs': crs_uri,
            'file_format': 'Geodatabase',
            'geometry_type': geometry_type,
            'bounding_box': bbox,
            'total_area_km2': total_area_km2,
            'spatial_resolution': spatial_resolution,
            'folder_size': f"{folder_size} MB",
            'wkt_outline': wkt_outline if include_wkt else None
        }
        metadata.append(file_metadata)

    except Exception as e:
        print(f"Could not read geodatabase {root}: {e}")



### Executing the code for Part 1

Run option 1 to process the metadata without extracting the WKT polygon outline
    Run option 2 to obtain the outline. Review the simplify_tolerance value and update if needed. `.01` can be used for decimal degrees.

In [ ]:
# Option 1 Exclude WKT outline from the metadata
extract_metadata(root_directory, include_wkt=False)

In [ ]:
# Option 2: Include WKT outline in the metadata
extract_metadata(root_directory, simplify_tolerance=.001, include_wkt=True)

## Part 2: Attribute Tables

This function will read the attribute table fields and write them to a CSV in a defined directory.

In [ ]:
def extract_attribute_table_info(root_directory, output_dir):
    # Supported vector formats by GeoPandas
    vector_formats = {
        '.shp': 'Shapefile',
        '.geojson': 'GeoJSON'
    }

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Walk through the directory and its subdirectories
    for root, _, files in os.walk(root_directory):
        for filename in files:
            # Get the file extension
            file_ext = os.path.splitext(filename)[1].lower()

            # Construct the full file path
            filepath = os.path.join(root, filename)

            # Check if the file is a recognized vector format
            if file_ext in vector_formats:
                try:
                    # Read the vector file with GeoPandas
                    gdf = gpd.read_file(filepath)

                    # Extract field information
                    field_info = []
                    for column in gdf.columns:
                        field_metadata = {
                            'Field Name': column,
                            'Data Type': str(gdf[column].dtype),
                            'Unique Values': gdf[column].nunique(),
                            'Null Values': gdf[column].isnull().sum(),
                            'Definition' : '',
                            'Definition Source' : ''
                        }
                        field_info.append(field_metadata)

                    # Convert the field information to a DataFrame
                    field_df = pd.DataFrame(field_info)

                    # Create the output CSV filename
                    output_csv = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_fields.csv")

                    # Save the DataFrame to a CSV file
                    field_df.to_csv(output_csv, index=False)

#                     print(f"Field information extracted for {filename}. CSV saved to {output_csv}")

                except Exception as e:
                    print(f"Could not read {filename}: {e}")

In [ ]:
# Extract attribute table information
extract_attribute_table_info(root_directory, output_directory)